In [ ]:
!pip install numpy==1.19.5
!pip install pandas==1.3.1  
!pip install tensorflow-gpu==2.5.0
!pip install tensorflow-datasets==4.4.0
!pip install tensorflow-hub==0.12.0  
!pip install giotto-tda==0.5.1
!pip install keras==2.7.0 
!pip install keras-flops==0.1.2
!pip install keras-tcn==3.3.0
!pip install matplotlib==3.5.0
!pip install jupyterlab==3.0.16
!pip install torch==1.10.0             
!pip install torchvision==0.11.1
!pip install tqdm==4.61.2
!pip install pytorch2keras==0.2.4 
!pip install onnx==1.8.0              
!pip install onnx-tf==1.8.0
!pip install onnx2keras==0.0.24
!pip install librosa==0.8.1
!pip install kapre==0.2.0
!pip install scikit-learn==0.24.2             
!pip install scipy==1.7.0

In [ ]:
import os
import warnings
warnings.filterwarnings('ignore')
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"
os.environ["CUDA_VISIBLE_DEVICES"]="1"
import pandas as pd
import numpy as np
from gtda.time_series import SlidingWindow
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.keras.backend import set_session
config = tf.compat.v1.ConfigProto() 
config.gpu_options.allow_growth = True  
config.log_device_placement = True  
sess2 = tf.compat.v1.Session(config=config)
set_session(sess2) 
import tensorflow_datasets as tfds
from tensorflow.keras.models import load_model
from tensorflow.keras.applications import imagenet_utils
from tensorflow.keras.preprocessing.image import img_to_array
from tensorflow.keras.preprocessing.image import load_img

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, decode_predictions, preprocess_input
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import matplotlib.pyplot as plt
inputShape = (224, 224)
preprocess = imagenet_utils.preprocess_input

In [ ]:
dataset_name = "cifar10"
(ds_train, ds_test), ds_info = tfds.load(
    dataset_name, split=["train", "test"], with_info=True, as_supervised=True
)
NUM_CLASSES = ds_info.features["label"].num_classes

IMG_SIZE = 224
batch_size = 1
size = (IMG_SIZE, IMG_SIZE)
ds_train = ds_train.map(lambda image, label: (tf.image.resize(image, size), label))
ds_test = ds_test.map(lambda image, label: (tf.image.resize(image, size), label))

def input_preprocess(image, label):
    label = tf.one_hot(label, NUM_CLASSES)
    return image, label


ds_train = ds_train.map(
    input_preprocess, num_parallel_calls=tf.data.AUTOTUNE
)
ds_train = ds_train.batch(batch_size=batch_size, drop_remainder=True)
ds_train = ds_train.prefetch(tf.data.AUTOTUNE)

ds_test = ds_test.map(input_preprocess)
ds_test = ds_test.batch(batch_size=batch_size, drop_remainder=True)

In [ ]:
model_path = input("model_path")
model  = tf.keras.models.load_model(model_path)

In [ ]:
from tensorflow.keras.losses import MSE
import tensorflow as tf


def fgsm_attack(model, image, label, eps):
  image = tf.cast(image, tf.float32)
  with tf.GradientTape() as tape:
    tape.watch(image)
    pred = model(image)
    loss = MSE(label, pred)
    gradient = tape.gradient(loss, image)
    signedGrad = tf.sign(gradient)
    adversary = (image + (signedGrad * eps)).numpy()
    return adversary

In [ ]:
def preprocess_img(image):
    image = tf.cast(image, tf.float32)
    image = tf.image.resize(image, (224, 224))
    image = preprocess_input(image)
    image = image[None, ...]
    
    return image

In [ ]:
count = 0
for image, label in ds_test:
  imageLabel = np.array(label).argmax()
  imagePred = model.predict(image)
  imagePred = imagePred.argmax()
  print("image label", imageLabel)
  print("image pred", imagePred)
  adversary = fgsm_attack(model,image, label, eps=0.2)
  pred = model.predict(adversary)
  adversaryPred = pred[0].argmax()
  print("adversaryPred", adversaryPred)
  if imagePred == adversaryPred:
    count += 1

print("Adversarial accuracy : " count / len(ds_test)) # no of instances where adversarial prediction matches with the model prediction / total test samples